In [1]:
#@Time:2021/4/25 10:27
#@Author: 戴 妍 晴
#@File:3.py
#@Software:PyCharm
#coding:utf-8
###python3.5
import pandas as pd
global N
pcb = []
sjf_pcb= []

In [2]:
#输入进程数据
def inPcbFromInput():
    global N
    N = int(input("请输入进程数："))
    for i in range(N):
        print("****************************************")
        pName = str(input("请输入进程编号 ："))                 #进程编号
        inTime = int(input("请输入进入时间："))               #进入时间
        serverTime = int(input("请输入服务时间："))           #服务时间
        #数据按顺序存放到列表中 startTime=开始时间 finishTime=完成时间 zzTime=周转时间 dqzzTime=带权周转时间
        #          进程编号 进入时间    服务时间 开始  完成 就绪队列 周转 带权 等待
        pcb.append([pName, inTime, serverTime,    0,     0,    "",     0,  0,  0])


In [3]:
#读取表格中的进程数据
def inPcbFromTable():
    global N
    Data = pd.read_excel(r'3.xlsx')  #读入表格数据
    print(Data)
    N = Data.shape[0]   #获取进程数
    print(N)
    for row in Data.itertuples():   #循环表格的每一行
#         print(row[1],row[2],row[3])
        #          进程编号 到达时间 服务时间 开始 完成 就绪队列 周转 带权 等待
        pcb.append([row[1] ,row[2] ,  row[3] , 0,    0 , "",    0,  0,  0])

In [4]:
#先来先服务
def fcfs():
    global N
    pcb.sort(key=lambda x:x[1], reverse=False)  #升序排序   x:x[1] 按照进入时间排序
    for i in range(N):
        if(i==0):
            startTime = int(pcb[i][1]) # 到达时间 = 开始时间
            pcb[i][3] = startTime  
            pcb[i][4] = startTime + int(pcb[i][2])   #完成 =开始+服务
            s = ""
            for x in range(i+1,N):
                s = s + " " +  str(pcb[x][0])
            pcb[0][5] = s
        else:
            if pcb[i-1][4] >= pcb[i][1]:
                pcb[i][3] = pcb[i-1][4]  #开始 = 上一个的完成
            else:
                pcb[i][3] = pcb[i][1]

            pcb[i][4] = pcb[i][3] + int(pcb[i][2])   #完成 =开始+服务
            s = ""
            for x in range(i+1,N):
                s = s + " " +  str(pcb[x][0])
            pcb[i][5] = s

    print("Process Num.    Start Time   End Time   Ready Queue")
    for i in range(N):
        print(f"{pcb[i][0]: ^15}",f"{int(pcb[i][3]): ^15}",f"{int(pcb[i][4]): ^10}",f"{pcb[i][5]: ^5}")
    sumTAT = 0
    sumWTAT = 0
    sumWT = 0
    for i in range(N):     #计算周转时间和带权周转时间
        pcb[i][6] = float(pcb[i][4]) - int(pcb[i][1])    #周转时间
        pcb[i][7] = float(pcb[i][6]) / int(pcb[i][2])    #带权周转时间
        pcb[i][8] = int(pcb[i][3] - pcb[i][1])     #等待时间=开始-到达
        sumTAT = sumTAT + pcb[i][6]
        sumWTAT = sumWTAT + pcb[i][7]
        sumWT = sumWT + pcb[i][8]
        
    #讲结果输出出来
    pcb.sort(key=lambda x:x[0], reverse=False)     ##升序排序  按照进程名排序
    print()

    print("Process Num.  Turnarround Time   WeightedTurnarround Time    Waiting Time")
    
    for i in range(N):
        print(f"{pcb[i][0]: ^15}",f"{int(pcb[i][6]): ^15}",f"{round(float(pcb[i][7]),3): ^30}",f"{int(pcb[i][8]): ^15}")
    print()

    print("Algorithm    Average TAT   Average WTAT   Average WT")
    print(f"{'FCFS': ^15}",f"{float(sumTAT/N): ^15}",f"{round(float(sumWTAT/N),3): ^10}",f"{float(sumWT/N): ^15}")
      

In [5]:
#短作业优先
def sjf():
    sjf_pcb=pcb
    temp_pcb = pcb[1:len(sjf_pcb)]   #切片 临时存放后备队列  len(sjf_pcb)获取长度。
    temp_pcb.sort(key=lambda x:x[2], reverse=False)  ##对后背队列 按照服务时间排序
    sjf_pcb[1:len(sjf_pcb)]=temp_pcb
    i = 0
    for i in range(N):
        if(i==0):
            startTime = int(sjf_pcb[0][1]) # 到达时间 = 开始时间
            sjf_pcb[0][3] = startTime
            sjf_pcb[0][4] = startTime + int(sjf_pcb[0][2])#完成 =开始+服务
            s = ""
            for x in range(i+1,N):
                s = s + " " +  str(sjf_pcb[x][0])

            sjf_pcb[0][5] = s
        else:
            startTime = sjf_pcb[i-1][4]  #开始 = 上一个的完成
            sjf_pcb[i][3] = startTime
            sjf_pcb[i][4] = startTime + int(sjf_pcb[i][2])  #完成 =开始+服务
            s = ""
            for x in range(i+1,N):
                s = s + " " +  str(sjf_pcb[x][0])
            sjf_pcb[i][5] = s
                

    print("Process Num.    Start Time   End Time   Ready Queue")
    for i in range(N):
        print(f"{sjf_pcb[i][0]: ^15}",f"{int(sjf_pcb[i][3]): ^15}",f"{int(sjf_pcb[i][4]): ^10}",f"{sjf_pcb[i][5]: ^5}")
    sumTAT = 0
    sumWTAT = 0
    sumWT = 0
    for i in range(N):
        sjf_pcb[i][6] = int(sjf_pcb[i][4]) - int(sjf_pcb[i][1])    #周转时间
        sjf_pcb[i][7] = float(sjf_pcb[i][6]) / int(sjf_pcb[i][2])    #带权周转时间
        sjf_pcb[i][8] = int(sjf_pcb[i][3] - sjf_pcb[i][1])     #等待时间=开始-到达
        sumTAT = sumTAT + sjf_pcb[i][6]
        sumWTAT = sumWTAT + sjf_pcb[i][7]
        sumWT = sumWT + sjf_pcb[i][8]
    #输出结果
    pcb.sort(key=lambda x:x[0], reverse=False)     ##升序排序  按照进程名排序
    print()
    print("Process Num.  Turnarround Time   WeightedTurnarround Time    Waiting Time")
    for i in range(N):
        print(f"{sjf_pcb[i][0]: ^15}",f"{int(sjf_pcb[i][6]): ^15}",f"{round(float(sjf_pcb[i][7]),3): ^30}",f"{int(sjf_pcb[i][8]): ^15}")
    print()
    print("Algorithm    Average TAT   Average WTAT   Average WT")
    print(f"{'SJF': ^15}",f"{float(sumTAT/N): ^15}",f"{round(float(sumWTAT/N),3): ^10}",f"{float(sumWT/N): ^15}")
        

In [6]:
if __name__ == '__main__':
    while 1:
        x = int(input("请选择要进行的算法过程：\n 1-FCFS \n 2-SJF \n"))
        
        y = int(input("请选择输入进程数据方式：\n 1-键盘输入数据 \n 2-读入表格数据 \n"))
        if y==1:
            inPcbFromInput()  #键盘输入
        else:
            inPcbFromTable()   #读入表格
        if x == 1:
            fcfs() #先来先服务
            break
        elif x == 2:
            sjf() #短作业优先
            break
        else:
            print("输入错误")


请选择要进行的算法过程：
 1-FCFS 
 2-SJF 
2
请选择输入进程数据方式：
 1-键盘输入数据 
 2-读入表格数据 
2
   Process Num  Arrival Time     CPU Burst
0            1                0          3
1            2                1         12
2            3                2          9
3            4                3         15
4
Process Num.    Start Time   End Time   Ready Queue
       1               0            3       3 2 4
       3               3            12      2 4 
       2              12            24       4  
       4              24            39          

Process Num.  Turnarround Time   WeightedTurnarround Time    Waiting Time
       1               3                     1.0                      0       
       2              23                    1.917                    11       
       3              10                    1.111                     1       
       4              36                     2.4                     21       

Algorithm    Average TAT   Average WTAT   Average WT
      SJF           